### 1. Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма) ###

In [36]:
import pprint
from selenium import webdriver          #Основной элемент
from selenium.webdriver.support.ui import Select   #Поле множественного выбора
from selenium.webdriver.common.keys import Keys    #Клавиши клавиатуры
import time
from pymongo import MongoClient, UpdateOne
import random

client = MongoClient('')
inbox = client.crawler.inbox

driver = webdriver.Chrome()
driver.get('https://mail.google.com/?ui=html')
assert 'Gmail' == driver.title
    
elem = driver.find_element_by_id("identifierId")
elem.send_keys('katekovna')
elem.send_keys(Keys.RETURN)
time.sleep(2)
elem = driver.find_element_by_name('password')
elem.send_keys('')
elem.send_keys(Keys.RETURN)
time.sleep(5)

base_xpath = '/html/body/table[2]/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/table[4]/tbody/tr/td/table[1]/tbody/'
update_list = []
for i in range(1,51):
    elem = driver.find_element_by_xpath(base_xpath[:59] + 'form/table[2]/tbody/tr[' + str(i) + ']/td[3]/a')
    elem.click()
    letter = {}
    letter['from'] = driver.find_element_by_xpath(base_xpath + 'tr[1]/td[1]').text
    letter['subject'] = driver.find_element_by_xpath(base_xpath[:59] + 'table[3]/tbody/tr/td/h2/font/b').text
    letter['date'] = driver.find_element_by_xpath(base_xpath + 'tr[1]/td[2]').text
    letter['text'] = driver.find_element_by_xpath(base_xpath + 'tr[4]').text
    letter['id'] = letter['from'] + '_' + letter['date']
    if i == 2:
        print(letter)
        
    update_list.append(UpdateOne({"id": letter['id']}, {"$set": letter}, upsert=True))
    driver.back()
    time.sleep(random.random())  # Не будем совсем уж палится, введём рандомную задержку между кликами
print(inbox.bulk_write(update_list).bulk_api_result)
pprint.pprint(inbox.find_one())
driver.quit()

{'from': '  Codecademy<learn@codecademy.com>', 'subject': "There's a better path", 'date': 'Mon, Jul 15, 2019 at 11:18 PM', 'text': "    If you can’t see this email - View it in your browser.\n                Web Development Path Update!\n  This week, we’ve launched updates to improve one of our most popular paths: the Web Development Path. If you’re unfamiliar, Paths are a more guided learning journey than our one off courses. You choose what you want (to learn a skill, build a thing, get qualified for a promotion) and we guide you through different languages, projects, and quizzes to get you to your desired outcome.\n  The Web Development path will teach you how build polished websites and web applications. You'll learn how to develop applications locally, and work on real-world style projects which tie together everything you've learned.\n  Generally, Paths are a Pro only feature but the first milestone in the Web Development Path is free and this update has added an additional less

### 2. Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары ###

In [87]:
import json
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

client = MongoClient('')
hits = client.crawler.hits

driver = webdriver.Chrome()
driver.get('https://www.mvideo.ru/')
assert 'М.Видео' in driver.title

time.sleep(2)

element = driver.find_element_by_xpath('/html/body/div/div/div[1]')
actions = ActionChains(driver)
actions.move_to_element(element).click().perform()

buttons = driver.find_elements_by_class_name('sel-hits-button-next')
driver.execute_script("arguments[0].scrollIntoView();", buttons[1])

time.sleep(2)

for _ in range (0,5):
    driver.execute_script("arguments[0].click();", buttons[1])
    time.sleep(3)
    
elements = driver.find_elements_by_class_name('sel-product-tile-title')
keys_to_store = ['productCategoryName', 'productName', 'productPriceLocal' , 'productVendorName', 'productId' ]
update_list = []
for element in elements[:19]:
    to_db = {key:value for key,value in json.loads(element.get_attribute('data-product-info')).items() if key in keys_to_store}
    update_list.append(UpdateOne({"id": to_db['productId']}, {"$set": to_db}, upsert=True))    

print(hits.bulk_write(update_list).bulk_api_result)
pprint.pprint(hits.find_one())
driver.quit()

{'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 19, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': [{'index': 0, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa8c')}, {'index': 1, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa8d')}, {'index': 2, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa8e')}, {'index': 3, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa8f')}, {'index': 4, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa90')}, {'index': 5, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa91')}, {'index': 6, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa92')}, {'index': 7, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa93')}, {'index': 8, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa94')}, {'index': 9, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa95')}, {'index': 10, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa96')}, {'index': 11, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa97')}, {'index': 12, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa98')}, {'index': 13, '_id': ObjectId('5d2ddc4a12d9f43b7ff5aa99')}, {'index': 14, '_id': ObjectId('5